<a href="https://colab.research.google.com/github/mlaricobar/WebScrapingCourse/blob/master/DMC2019I_NOT3_Web_Scraping_AdondeVivir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping de Adondevivir

En esta oportunidad intentaremos scrapear los datos de un sitio web tal como [AdondeVivir.com](https://www.adondevivir.com/), que es un portal inmobiliario donde encontraremos departamentos, casas, terrenos en alquiler y venta.

<img src="https://raw.githubusercontent.com/mlaricobar/Machine-Learning-Course/84d066f02eb8a3b0c342c2c1ad182657dc07f972/images/adondevivir.png" width="700">

Se nos encomendado una tarea muy importante que consiste en investigar un poco sobre el sector Inmobiliario. Por ejemplo, queremos entender el comportamiento del precio de los inmuebles, que se encuentran en alquiler en Lima, en base a su geolocalización. 

1. ¿Podemos hacer esto?
2. ¿Tenemos data para hacerlo?
3. ¿Una vez que tengamos la data, que podemos hacer?

# Librerías de Web Scraping

En esta sesión, haremos uso de las siguientes librerías para poder realizar Web Scraping:

1. [Requests](http://docs.python-requests.org/en/master/): Esta librería nos permitirá hacer peticiones HTTP.
2. [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): Módulo para analizar documentos HTML. Esta biblioteca crea un árbol con todos los elementos del documento y puede ser utilizado para extraer información.

# Instalación de Librerías

In [0]:
# Librería Requests
!pip install requests

In [0]:
# Librería BeautifulSoup
!pip install beautifulsoup4

Una vez instalada las librerías necesarias, hagamos las pruebas correspondientes para verificar que todo esté correctamente instalado.

In [0]:
# importar la librería requests
import requests

In [0]:
# importar la librería BeautifulSoup
from bs4 import BeautifulSoup

# Pasos para realizar Web Scraping

### 1. Hacer una petición a la Web

In [0]:
# Definir Url de la página que vamos a Scrapear
url = "https://www.adondevivir.com/departamentos-en-alquiler-en-lima.html"

#### Usando la librería Requests

In [0]:
# Hagamos una petición a la url usando el método get del módulo requests
page = requests.get(url)

In [0]:
# Pintemos el contenido de la respuesta usando la propiedad content
print(page.content)

In [0]:
# Instanciemos el objeto de la clase BeautifulSoup usando como parámetro el contenido de la respuesta obtenida de la petición
soup = BeautifulSoup(markup=page.content, features='html.parser')

In [0]:
# ¿Qué tipo de dato es soup?
type(soup)

In [0]:
# Mostrar el html interpretado por BeautifulSoup
soup.prettify

In [0]:
# Mostremos el título del html
soup.title

Si queremos extraer todos los inmuebles de esta página, definamos una estrategia para Scrapear. **Volvamos a la ppt**.

Si se han dado cuenta al inspeccionar la página, el elemento que contiene a la lista de inmuebles (sección) de una página tiene una clase llamada `list-card-container`. Aprovechemos ese atributo para poder identificar a cada sección usando el método `find`.

In [0]:
# Encontremos el elemento que contenga los avisos de los inmuebles usando el método find
section_ = soup.find(name="div", attrs={"class": "list-card-container"})

In [0]:
# Mostremos el contenido de este elemento
section_

In [0]:
# Veamos como podemos extraer el identificador del inmueble y la url donde encontraremos más detalle
property_list = [{"id": s.get("data-id"), "url": "https://www.adondevivir.com" + s.get("data-to-posting")} for s in section_.find_all(name="div", attrs={"class": "posting-card"})]

#### Construcción del Web Crawler

A continuación construiremos nuestro Web Crawler que va a recorrer el página de Inmuebles de Lima con el fin de extraer las urls de detalle de los mismos.

In [0]:
data_input_list = []
for i in range(1, 11):
    url = "https://www.adondevivir.com/departamentos-en-venta-en-lima-pagina-{0}.html".format(i)
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(markup=page.content, features='html.parser')
    section_ = soup.find(name="div", attrs={"class": "list-card-container"})
    window_list = section_.find_all(name="div", attrs={"class": "posting-card"})
    property_list = [{"id": s.get("data-id"), "url": "https://www.adondevivir.com" + s.get("data-to-posting")} for s in window_list]
    
    print("\t Cantidad de Inmuebles: {0}".format(len(property_list)))
    data_input_list += property_list

<img src="https://raw.githubusercontent.com/mlaricobar/Machine-Learning-Course/84d066f02eb8a3b0c342c2c1ad182657dc07f972/images/see-Firefox.jpg" width="400">

Pero ... ¿Podemos aprovechar para extraer datos de estos elementos?

#### Web Crawler con Web Scrapper

In [0]:
# Obtenemos un elemento de los 25 obtenidos. A este elemento le denominaremos window como variable
window_tmp = window_list[0]

In [0]:
# Al analizar nos damos cuenta de que este elemento window tiene 2 columnas, en las que se encuentran datos de imagenes, precios y detalle del inmueble
# En este caso identificamos la sección de la primera columna (imagenes y precio)
first_column_section = window_tmp.find(name="div", attrs={"class": "first-column"})

In [0]:
# Identificamos el elemento donde se encuentran las imágenes del inmueble
gallery_section = first_column_section.find(name="div", attrs={"class": "posting-gallery-container"})

In [0]:
# Identificamos el elemento donde se encuentran los precios del inmueble
price_section = first_column_section.find(name="div", attrs={"class": "posting-price-container"})

In [0]:
# Identificar los elementos que contienen las imagenes
img_element_list = gallery_section.find_all(name="img")

In [0]:
# Obtener las urls de las imagenes
img_list = [i.get("src") for i in img_element_list if 'src' in i.attrs.keys()]
img_list

In [0]:
# obtener el texto del precio (si es un valor fijo o variable) al encontrar el elemento con la clase "from-price"
var_desde = price_section.find(name="span", attrs={"class": "from-price"}).get_text()

In [0]:
# obtener el valor del precio al encontrar el elemento con la clase "first-price"
var_first_price = price_section.find(name="span", attrs={"class": "first-price"}).get_text()

In [0]:
# obtener el valor del segundo precio del inmueble (en dolares) al encontrar el elemento con la clase "second-price"
var_second_price = price_section.find(name="span", attrs={"class": "second-price"}).get_text()

In [0]:
# Pintar los valores ya extraídos
var_desde, var_first_price, var_second_price

In [0]:
# En este caso identificamos la sección de la segunda columna (detalle)
second_column_section = window_tmp.find(name="div", attrs={"class": "second-column"})

In [0]:
# Dentro de esta columna, encontramos 2 secciones: Informacion de atributos y de publicacion
# Esta seccion es de atributos
info_section = second_column_section.find(name="div", attrs={"class": "posting-info-container"})

In [0]:
# Esta seccion es de publicacion
features_section = second_column_section.find(name="div", attrs={"class": "posting-features-container"})

In [0]:
# Obtenemos los elementos que contienen los atributos del inmueble
main_features_section = info_section.find(name="ul", attrs={"class": "main-features"})

In [0]:
# Obtenemos los valores de los atributos del inmueble
feature_list = [l.get_text().strip() for l in main_features_section.find_all(name="li")]

In [0]:
# Pintamos los valores obtenidos
feature_list

In [0]:
# Obtener la descripción del aviso del inmueble
var_description = info_section.find(name="div", attrs={"class": "posting-description"}).get_text().strip()

In [0]:
# Obtener el titulo del aviso
var_title = info_section.find(name="h3", attrs={"class": "posting-title"}).get_text().strip()

In [0]:
# Obtener la dirección física del inmueble
var_address = info_section.find(name="span", attrs={"class": "posting-location"}).get_text(";").strip()

#####  Función de Web Scrapper a nivel de Window

In [0]:
# Esta función recibirá como entrada el elemento window definido y obtendrá como resultado los datos que contiene

def get_window(window_tmp):
    var_id = window_tmp.get("data-id")
    var_url = "https://www.adondevivir.com" + window_tmp.get("data-to-posting")
    print("\t Url: {0}".format(var_url))
    
    first_column_section = window_tmp.find(name="div", attrs={"class": "first-column"})
    gallery_section = first_column_section.find(name="div", attrs={"class": "posting-gallery-container"})
    price_section = first_column_section.find(name="div", attrs={"class": "posting-price-container"})
    img_element_list = gallery_section.find_all(name="img")
    
    try:
        var_img_list = [i.get("src") for i in img_element_list if 'src' in i.attrs.keys()]
    except AttributeError:
        var_img_list = []
    
    try:
        var_desde = price_section.find(name="span", attrs={"class": "from-price"}).get_text()
    except AttributeError:
        var_desde = ""
    
    try:
        var_first_price = price_section.find(name="span", attrs={"class": "first-price"}).get_text()
    except AttributeError:
        var_first_price = ""
    
    try:
        var_second_price = price_section.find(name="span", attrs={"class": "second-price"}).get_text()
    except AttributeError:
        var_second_price = ""
    
    second_column_section = window_tmp.find(name="div", attrs={"class": "second-column"})
    info_section = second_column_section.find(name="div", attrs={"class": "posting-info-container"})
    features_section = second_column_section.find(name="div", attrs={"class": "posting-features-container"})
    main_features_section = info_section.find(name="ul", attrs={"class": "main-features"})
    
    try: 
        var_feature_list = [l.get_text().strip() for l in main_features_section.find_all(name="li")]
    except AttributeError:
        var_feature_list = []
    
    try:
        var_description = info_section.find(name="div", attrs={"class": "posting-description"}).get_text().strip()
    except AttributeError:
        var_description = ""
    
    try:
        var_title = info_section.find(name="h3", attrs={"class": "posting-title"}).get_text().strip()
    except AttributeError:
        var_title = ""
    
    try:
        var_address = info_section.find(name="span", attrs={"class": "posting-location"}).get_text(";").strip()
    except AttributeError:
        var_address = ""
    
    
    
    window_data =  {
        "id": var_id,
        "url": var_url,
        "img_list": var_img_list,
        "from": var_desde,
        "first_price": var_first_price,
        "second_price": var_second_price,
        "feature_list": var_feature_list,
        "description": var_description,
        "title": var_title,
        "address": var_address
    }
    
    unit_data = get_unit_data(var_url)
    
    window_data.update(unit_data)
    
    return window_data

In [0]:
# Un ejemplo del retorno de la función
get_window(window_tmp)

In [0]:
import re

#####  Función de Web Scrapper a nivel de Detalle de Inmueble

In [0]:
def get_unit_data(url):
    page = requests.get(url)
    soup = BeautifulSoup(markup=page.content, features='html.parser')
    soup.find(name="div", attrs={"class": "mv-gd-widget-20"})

    javascript_code = [s for s in soup.find_all(name="script") if "avisoInfo" in s.get_text()][0].get_text()
    try:
        var_latitude = re.search(r"'mapLat': (.+),", javascript_code).group(1).strip()
    except AttributeError:
        var_latitude = ""
    try:
        var_longitude = re.search(r"'mapLng': (.+),", javascript_code).group(1).strip()
    except AttributeError:
        var_longitude = ""
    try:
        var_operation_type = re.search(r"'operationType' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_operation_type = ""
    try:
        var_property_type = re.search(r"'propertyType' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_property_type = ""
    try:
        var_listing_category = re.search(r"'listingCategory' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_listing_category = ""
    try:
        var_neighborhood = re.search(r"'neighborhood' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_neighborhood = ""
    try:
        var_city = re.search(r"'city' : '(.+)',", javascript_code).group(1).strip()
    except AttributeError:
        var_city = ""
    try:
        var_province = re.search(r"'province' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_province = ""
    try:
        var_sell_price = re.search(r"'sellPrice' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_sell_price = ""
    try:
        var_rental_price = re.search(r"'rentalPrice' : '(.*)',", javascript_code).group(1).strip()
    except AttributeError:
        var_rental_price = ""
    
    return {
        "latitude" : var_latitude,
        "longitude" : var_longitude,
        "operation_type" : var_operation_type,
        "property_type" : var_property_type,
        "listing_category" : var_listing_category,
        "neighborhood" : var_neighborhood,
        "city" : var_city,
        "province" : var_province,
        "sell_price" : var_sell_price,
        "rental_price" : var_rental_price
    }

In [0]:
# Crawleamos 200 páginas y obtengamos los datos
data_input_list = []
for i in range(1, 201):
    url = "https://www.adondevivir.com/departamentos-en-venta-en-lima-pagina-{0}.html".format(i)
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(markup=page.content, features='html.parser')
    section_ = soup.find(name="div", attrs={"class": "list-card-container"})
    window_list = section_.find_all(name="div", attrs={"class": "posting-card"})
    property_list = [get_window(s) for s in window_list]
    
    print("\t Cantidad de Inmuebles: {0}".format(len(property_list)))
    data_input_list += property_list

In [0]:
import pandas as pd

In [0]:
# Carguemos como un dataframe de Pandas
df = pd.DataFrame(data_input_list)

In [0]:
# Exportemos el dataframe a un csv
df.to_csv("data_apartments.csv", index=False)

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!cp data_apartments.csv /gdrive/'My Drive'

#### Lectura y Análisis de Inmuebles Scrapeados

In [0]:
df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)

#### Construcción de un Modelo de Predicción de Precios

**Definición de Target**

In [0]:
df = df.loc[df["first_price"].apply(lambda p: ('usd' not in p.lower()) and (p.lower().strip() != "")), :].reset_index(drop=True)

In [0]:
df.shape

In [0]:
df["first_price"] = df["first_price"].apply(lambda p: float(re.search(r'S/ (.*)', p).group(1).replace(",", "")))

In [0]:
df[["first_price"]].describe()

In [0]:
df.loc[df["first_price"] < 10000000, "first_price"].hist(bins=50)

**Definición de Features**

In [0]:
df["property_type"].value_counts()

In [0]:
df = df.loc[df["property_type"] == "Departamento"].reset_index(drop=True)

In [0]:
df["operation_type"].value_counts()

In [0]:
df["listing_category"].value_counts()

In [0]:
df["from"].value_counts()

In [0]:
df["len_feature_list"] = df["feature_list"].apply(lambda l: len(l))

In [0]:
df["len_feature_list"].value_counts()

In [0]:
def get_data_from_pattern(pattern_, list_):
  value = None
  for val in list_:
    if re.search(pattern_, val.lower()):
      value = re.search(pattern_, val.lower()).group(1).strip()
      return value
  return value

In [0]:
df["unidades"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*) unidades', l))

In [0]:
df["dormitorios"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*) dormitorios?', l))

In [0]:
df["m2_techado"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*)\s*m² techados', l))

In [0]:
df["m2_total"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*)\s*m² totales', l))

In [0]:
df["banhos"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*) baños?', l))

In [0]:
df["estacionamientos"] = df["feature_list"].apply(lambda l: get_data_from_pattern(r'(.*) estacionamientos?', l))

In [0]:
df.head()

In [0]:
features = ["len_feature_list", "dormitorios", "m2_techado", "m2_total", "banhos", "estacionamientos", "first_price"]
predictors = ["len_feature_list", "dormitorios", "m2_techado", "m2_total", "banhos", "estacionamientos"]
target = "first_price"

In [0]:
for feature in features:
  df[feature] = df[feature].astype(float)

In [0]:
df[features].info()

In [0]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [0]:
X = df[predictors]
y = df["first_price"]

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30)

In [0]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

In [0]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
n_estimators = 100

In [0]:
watchlist = [d_train, d_valid]
model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

¿Que pasó? ¿Por qué el valor del error sigue siendo elevado?


<img src="https://raw.githubusercontent.com/mlaricobar/Machine-Learning-Course/84d066f02eb8a3b0c342c2c1ad182657dc07f972/images/why-meme.png" width="400">

In [0]:
y_pred = model.predict(x_valid, num_iteration=model.best_iteration)

In [0]:
x_valid["y_real"] = y_valid
x_valid["y_predict"] = y_pred
x_valid["y_predict"] = round(x_valid["y_predict"], 0)

In [0]:
x_valid

# Visualizar los inmuebles en un Mapa

Para poder visualizar las coordenadas de todos los inmuebles extraídos por el proceso de Web Scraping, haremos uso de [Mapbox](https://www.mapbox.com/).

Mapbox es un proveedor de mapas on-line realizados por encargo para páginas webs como **Foursquare, Pinterest, Evernote, Financial Times, EThe Weather Channel y Uber Tecnologías**. Desde 2010, ha expandido rápidamente su nicho de mapas como respuesta a la limitada elección que ofrecen otros proveedores como Google Maps y OpenStreetMap.

<br>
<img src="https://raw.githubusercontent.com/mlaricobar/Machine-Learning-Course/84d066f02eb8a3b0c342c2c1ad182657dc07f972/images/mapbox.png" width="400">

Para poder usar mapbox, necesitamos de un token que se genera al crear una cuenta.

Para poder usar esta herramienta dentro de Jupyter, debemos instalar el módulo [mapboxgl](https://github.com/mapbox/mapboxgl-jupyter).

In [0]:
!pip install mapboxgl

In [0]:
# Importar los métodos del módulo mapboxgl
from mapboxgl.utils import *
from mapboxgl.viz import *

In [0]:
# Definir token de Mapbox
token = "pk.eyJ1IjoibWxhcmljb2JhciIsImEiOiJjanY4eGNnazEwdmxoNDRudDl6NTh5ajE2In0.lmuCjr7vHr0DxFWXn3yJWg"

In [0]:
# Lectura de un dataset de inmuebles ya scrapeados
df_geo = pd.read_csv("https://github.com/mlaricobar/WebScrapingCourse/blob/master/apartments.csv", encoding="latin-1")
df_geo.head()

In [0]:
df.info()

In [0]:
df.dropna(subset=["price"], inplace=True)

In [0]:
# Crear un archivo geojson y exportarlo a paratir del DataFrame df
df_to_geojson(df_tmp, filename='apartments_geopoints.geojson',
              properties=['unit_id', 'title', 'price'], 
                     lat='latitude', lon='longitude', precision=3)

In [0]:
# Mostrar los puntos en el mapa
viz = CircleViz('apartments_geopoints.geojson', access_token=token, 
                radius = 2, center = (-77.021779, -12.093512), zoom = 11)
viz.show()

Ahora si queremos diferenciar los inmuebles por rangos de precios. Para ello, usemos los deciles de los valores de los precios.

Haremos uso del método `quantile` del DataFrame, que nos retorna el valor en el cuantil indicado.

¿Pero qué son los cuantiles?
Los cuantiles son puntos tomados a intervalos regulares de la función de distribución de una variable aleatoria.

In [0]:
measure = 'price'
color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(1,9)]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

In [0]:
# Plotear las coordenadas de los inmuebles y especificar la variable y los puntos de corte
viz = CircleViz('apartments_geopoints.geojson',
                access_token=token, 
                color_property = "price",
                color_stops = color_stops,
                radius = 2.5,
                stroke_width = 0.2,
                center = (-77.043322, -12.043980),
                zoom = 11,
                below_layer = 'waterway-label'
               )
viz.show()

In [0]:
# Definir otro estilo
viz.style='mapbox://styles/mapbox/dark-v9?optimize=true'
viz.label_color = 'hsl(0, 0%, 70%)'
viz.label_halo_color = 'hsla(0, 0%, 10%, 0.75)'
viz.show()